# Importando Bibliotecas

In [1]:
import numpy as np
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn import preprocessing


# Inicializando Dataframes

In [2]:
df = pd.read_csv('exame_cmc13_dados_teste.csv', sep=';')
df.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'user_id', 'age', 'isbn', 'rating',
       'book_title', 'book_author', 'year_of_publication', 'publisher',
       'img_l', 'Language', 'Category', 'city', 'state', 'country'],
      dtype='object')

# Tratando Dados

Vemos campos que não são mencionados e que, provavelmente, se referem apenas a identificações internas dos livros: 'Unnamed: 0.1' e 'Unnamed: 0'. Vamos excluí-los.

In [3]:
df = df.drop(['Unnamed: 0.1', 'Unnamed: 0'], axis=1)

Vamos começar a exluir colunas inúteis para a análise. Em um primeiro momento, podemos excluir as colunas de identificação do usuário:

In [4]:
df = df.drop(['user_id'], axis=1)

Agora, vamos filtrar as localidades. Para simplificar a análise, contaremos que leitores de um mesmo país têm gostos semelhantes, excluindo a necessidade de identificadores de cidades e estados. Isso também é permitido pelo fato de haver uma variedade de países:

In [5]:
df['country'].value_counts()

usa                      25623
canada                    3096
united kingdom             655
australia                  434
germany                    258
                         ...  
queensland, australia        1
pakistan                     1
england                      1
kenya                        1
burma                        1
Name: country, Length: 116, dtype: int64

Existem 116 países no dataset. Excluindo colunas referentes a localidades:

In [6]:
df = df.drop(['city', 'state'], axis=1)

Análise da linguagem dos livros:

In [7]:
df['Language'].value_counts()

en    21670
9     11125
Name: Language, dtype: int64

Veja que apenas duas classificações foram categorizadas: en (inglês) e 9 (provavelmente um placeholder ou erro de obtenção de dados). Como a análise desses dois classificadores não nos fornece tanta informação, é razoável excluir tal coluna.

In [8]:
df = df.drop(['Language'], axis=1)

Podemos excluir as colunas 'isbn' porque se refere a uma identificação do livro, redundante com o título, e 'img_l', pois, apesar de a capa certamente ser importante para a escolha de um livro, apenas o link da imagem não adiciona tanto à análise.

In [9]:
df = df.drop(['isbn', 'img_l'], axis=1)

Veja que as colunas de idade e ano de publicação estão sendo tratadas como floats. Convertendo ambas para int:

In [10]:
df['age'] = df['age'].astype(int)
df['year_of_publication'] = df['year_of_publication'].astype(int)

Entretanto, ainda devem existir colunas com valores nulos. Vamos contá-las:

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31684 entries, 0 to 32794
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   age                  31684 non-null  int32 
 1   rating               31684 non-null  int64 
 2   book_title           31684 non-null  object
 3   book_author          31684 non-null  object
 4   year_of_publication  31684 non-null  int32 
 5   publisher            31684 non-null  object
 6   Category             31684 non-null  object
 7   country              31684 non-null  object
dtypes: int32(2), int64(1), object(5)
memory usage: 1.9+ MB


Veja que apenas 31684 linhas de 'country' tem valores não nulos, contra 32795 das outras colunas. Como são apenas 1111 de quase 33000 dados, representando aproximadamente 3% do dataset, podemos excluí-los sem grande perda:

In [14]:
df = df.dropna()
print(df)

       age  rating                                         book_title  \
0       35       3                                 The Hours: A Novel   
1       34      10                  Seven Up (A Stephanie Plum Novel)   
2       33      10                                The Handmaid's Tale   
3       33       8                                       The Notebook   
4       44       0                                When the Wind Blows   
...    ...     ...                                                ...   
32790   59       0                                    Four Blind Mice   
32791   34       9                The Red Tent (Bestselling Backlist)   
32792   25       0                                      The Rainmaker   
32793   55       0  Morality for Beautiful Girls (No.1 Ladies Dete...   
32794   34       0                                     ANGELA'S ASHES   

                  book_author  year_of_publication                publisher  \
0          Michael Cunningham               

# Construção do modelo MLP

In [13]:
X_train = df.drop(['rating'], axis = 1)
Y_train = df['rating']

Encodificando as Strings para que o fit no modelo funcione (strings não têm valor de análise para o SKLearn, por isso precisam ser encodificadas):

In [14]:
le = preprocessing.LabelEncoder()
for column_name in X_train.columns:
    if X_train[column_name].dtype == object:
        X_train[column_name] = le.fit_transform(X_train[column_name])
    else:
        pass

,age,book_title,book_author,year_of_publication,publisher,Category,country
0,35,730,292,2002,94,30,111
1,34,551,188,2002,127,0,111
2,33,721,272,1989,42,13,111
3,33,770,307,1998,143,0,111
4,44,913,181,1999,145,30,111
...,...,...,...,...,...,...,...
32790,59,257,181,2002,74,0,111
32791,34,798,22,1998,96,30,111
32792,25,795,167,1996,31,30,111
32793,55,418,13,2002,5,30,111


Agora, vamos treinar o modelo:

In [15]:
clf = MLPClassifier(hidden_layer_sizes = 50, random_state = 1, max_iter = 300).fit(X_train, Y_train)

Carregando os dados de teste:

In [16]:
df_test = pd.read_csv('exame_cmc13_dados_teste.csv', sep=';')

Aplicando o mesmo tratamento feito aos dados de treino:

In [17]:
df_test = df_test.drop(['Unnamed: 0.1', 'Unnamed: 0', 'user_id', 'city', 'state', 'Language', 'isbn', 'img_l'], axis=1)
df_test['age'] = df_test['age'].astype(int)
df_test['year_of_publication'] = df_test['year_of_publication'].astype(int)

X_test = df_test.drop(['rating'], axis = 1)
Y_test = df_test['rating']

for column_name in X_test.columns:
    if X_test[column_name].dtype == object:
        X_test[column_name] = le.fit_transform(X_test[column_name])
    else:
        pass

Testando a eficácia do modelo projetado:

In [18]:
clf.score(X_test, Y_test)

0.588992224424455

Vamos testar qual o número de camadas que otimiza a classificação. Para isso, façamos uma lista com as porcentagens de acerto:

In [19]:
score = []

for i in range(1, 101):
    clf = MLPClassifier(hidden_layer_sizes = i, random_state = 1, max_iter = 300).fit(X_train, Y_train)
    score.append(clf.score(X_test, Y_test))

c:\Python310\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:699: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


In [ ]:
index_max = max(range(len(score)), key=score.__getitem__)
print('A quantidade de camadas que retorna a maior acurácia é', index_max+1,', com precisão', score[index_max])

A quantidade de camadas que retorna a maior acurácia é 18 , com precisão 0.5898460131117549


Finalmente, façamos o modelo com maior precisão:

In [ ]:
clf = MLPClassifier(hidden_layer_sizes = 18, random_state = 1, max_iter = 300).fit(X_train, Y_train)
clf.score(X_test, Y_test)

0.5898460131117549